In [1]:
import numpy as np
import pickle
import os
import random
from sklearn.preprocessing import LabelBinarizer

img_size = 32
num_channels = 3
num_classes = 10
num_file_train = 5
img_per_file = 10000
num_images_train = num_file_train * img_per_file

data_path = "CIFAR-10/"

In [2]:
def get_file_path(filename=""):
    return os.path.join(data_path, filename)

In [3]:
def unpickle(filename):
    file_path = get_file_path(filename)

    print("Loading data: " + file_path)

    with open(file_path, mode='rb') as file:
        data = pickle.load(file, encoding='bytes')

    return data

In [4]:
def convert_images(raw):
    raw_float = np.array(raw, dtype=float) / 255.0

    images = raw_float.reshape([-1, num_channels, img_size, img_size])

    images = images.transpose([0, 2, 3, 1])

    return images

In [5]:
def load_data(filename):
    data = unpickle(filename)
    raw_images = data[b'data']
    cls = np.array(data[b'labels'])

    images = convert_images(raw_images)

    return images, cls

In [6]:
def load_class_names():
    raw = unpickle(filename="batches.meta")[b'label_names']

    names = [x.decode('utf-8') for x in raw]

    return names

In [7]:
def load_training_data():

    images = np.zeros(shape=[num_images_train, img_size, img_size, num_channels], dtype=float)
    cls = np.zeros(shape=[num_images_train], dtype=int)

    # Begin-index for the current batch.
    begin = 0

    # For each data-file.
    for i in range(num_file_train):
        # Load the images and class-numbers from the data-file.
        images_batch, cls_batch = load_data(filename="data_batch_" + str(i + 1))

        # Number of images in this batch.
        num_images = len(images_batch)

        # End-index for the current batch.
        end = begin + num_images

        # Store the images into the array.
        images[begin:end, :] = images_batch

        # Store the class-numbers into the array.
        cls[begin:end] = cls_batch

        # The begin-index for the next batch is the current end-index.
        begin = end
        
    lb = LabelBinarizer()
    lb.fit(range(num_classes))

    return images, cls, lb.transform(cls)

In [8]:
def load_test_data():
    images, cls = load_data(filename="test_batch")

    lb = LabelBinarizer()
    lb.fit(range(num_classes))

    return images, cls, lb.transform(cls)

In [9]:
train_images, train_cls, train_label = load_training_data()
test_images, test_cls, test_label = load_test_data()

Loading data: CIFAR-10/data_batch_1
Loading data: CIFAR-10/data_batch_2
Loading data: CIFAR-10/data_batch_3
Loading data: CIFAR-10/data_batch_4
Loading data: CIFAR-10/data_batch_5
Loading data: CIFAR-10/test_batch


In [40]:
import tensorflow as tf

x = tf.placeholder(tf.float32, shape=[None,32,32,3])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

K = 32
L = 32
M = 32

W1 = tf.Variable(tf.truncated_normal([5,5,3,K], stddev = 0.1))
b1 = tf.Variable(tf.ones([K])/10)

W2 = tf.Variable(tf.truncated_normal([5,5,K,L], stddev = 0.1))
b2 = tf.Variable(tf.ones([L])/10)

W3 = tf.Variable(tf.truncated_normal([4,4,L,M], stddev = 0.1))
b3 = tf.Variable(tf.ones([M])/10)

N1 = 120
N2 = 72

W4 = tf.Variable(tf.truncated_normal([8*8*M,N1], stddev = 0.1))
b4 = tf.Variable(tf.ones([N1])/10)

W5 = tf.Variable(tf.truncated_normal([N1,N2], stddev = 0.1))
b5 = tf.Variable(tf.ones([N2])/10)

W6 = tf.Variable(tf.truncated_normal([N2,10], stddev = 0.1))
b6 = tf.Variable(tf.ones([10])/10)

y1 = tf.nn.relu(tf.nn.conv2d(x,W1,strides=[1,1,1,1], padding='SAME')+b1)
pool1 = tf.nn.max_pool(y1,ksize=[1,2,2,1],strides=[1,1,1,1], padding='SAME')
norm1 = tf.nn.lrn(pool1,depth_radius=5,bias=1.0, alpha=1e-4 ,beta=0.75)

y2 = tf.nn.relu(tf.nn.conv2d(norm1,W2,strides=[1,2,2,1], padding='SAME')+b2)
pool2 = tf.nn.max_pool(y2,ksize=[1,2,2,1],strides=[1,1,1,1], padding='SAME')
norm2 = tf.nn.lrn(pool2,depth_radius=5,bias=1.0, alpha=1e-4 ,beta=0.75)

y3 = tf.nn.relu(tf.nn.conv2d(norm2,W3,strides=[1,2,2,1], padding='SAME')+b3)
pool3 = tf.nn.max_pool(y3,ksize=[1,2,2,1],strides=[1,1,1,1], padding='SAME')
norm3 = tf.nn.lrn(pool3,depth_radius=5,bias=1.0, alpha=1e-4 ,beta=0.75)

yy = tf.reshape(norm3,shape=[-1,8*8*M])

y4 = tf.nn.relu(tf.matmul(yy,W4)+b4)
y5 = tf.nn.relu(tf.matmul(y4,W5)+b5)
y = tf.nn.softmax(tf.matmul(y5,W6)+b6)

cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.0001).minimize(cross_entropy)

In [41]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

loop = 5000
n = 128

for i in range(loop):
    index = range(n*i,n*(i+1))
    index = [j%num_images_train for j in index]
    batch_x = train_images[index]
    batch_y = train_label[index]
    sess.run(train_step,feed_dict={x: batch_x, y_: batch_y})
    
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: test_images, y_: test_label}))

sess.close()

ResourceExhaustedError: OOM when allocating tensor with shape[10000,32,32,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: Conv2D_32 = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_Placeholder_24_0_0/_13, Variable_128/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: add_58/_15 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_50_add_58", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunO